# **Chapter 74: Complete Financial Prediction System**

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Design the end‑to‑end architecture of a time‑series prediction system.
- Integrate data collection, feature engineering, model training, backtesting, and deployment into a cohesive pipeline.
- Implement a backtesting framework that respects temporal order and prevents look‑ahead bias.
- Deploy the model as a REST API and as a batch prediction service.
- Set up monitoring for model performance, data drift, and system health.
- Analyse historical performance and extract actionable lessons.
- Plan future improvements based on real‑world feedback.

---

## **74.1 System Architecture**

A complete financial prediction system is not just a Jupyter notebook with a model. It is a set of interconnected components that work together to deliver reliable, maintainable, and actionable predictions. Figure 74.1 illustrates the high‑level architecture of our NEPSE prediction system.

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│                 │     │                 │     │                 │
│  Data Sources   │────▶│  Data Ingestion │────▶│  Data Storage   │
│  (NEPSE CSV,    │     │  (daily ETL)    │     │  (Parquet, DB)  │
│   APIs, etc.)   │     │                 │     │                 │
└─────────────────┘     └─────────────────┘     └─────────────────┘
                                                           │
                                                           ▼
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│                 │     │                 │     │                 │
│  Monitoring &   │◀────│  Prediction     │◀────│  Feature Store  │
│  Alerting       │     │  Service        │     │  (engineered    │
│  (Chapter 73)   │     │  (REST/batch)   │     │   features)     │
└─────────────────┘     └─────────────────┘     └─────────────────┘
         │                       │
         ▼                       ▼
┌─────────────────┐     ┌─────────────────┐
│                 │     │                 │
│  Drift Detector │     │  Model Registry │
│  (concept/data) │     │  (MLflow)       │
└─────────────────┘     └─────────────────┘
```

The architecture consists of several layers:

1. **Data Layer**: Raw NEPSE data from CSV files (or APIs) is ingested and stored in a scalable format (Parquet) and optionally in a time‑series database for fast queries.
2. **Feature Layer**: A feature engineering pipeline transforms raw data into model‑ready features. The results are stored in a **feature store** for reuse across training and inference.
3. **Model Layer**: Models are trained, validated, and registered in a **model registry** (e.g., MLflow). The registry manages versions, metadata, and staging/production tags.
4. **Prediction Layer**: A prediction service serves the model via REST API (real‑time) or as a batch job (daily).
5. **Monitoring Layer**: System and model metrics are collected; alerts are triggered when anomalies or drifts are detected (Chapter 73).
6. **Orchestration**: Workflow managers (Airflow, Prefect) schedule and coordinate the entire pipeline.

In the following sections, we will implement each component, building on the code from previous chapters.

---

## **74.2 Data Collection Pipeline**

The data collection pipeline is responsible for acquiring new data, validating it, and storing it in a consistent format. For our NEPSE system, the primary source is a daily CSV file, but the design should be extensible to APIs.

```python
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from datetime import datetime
import requests
import io

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class NEPSEIngestion:
    """
    Handles ingestion of NEPSE data from CSV or API.
    
    For demonstration, we assume a local CSV file is updated daily.
    In production, you might pull from an API.
    """
    
    def __init__(self, data_dir: str = "./data/raw"):
        self.data_dir = Path(data_dir)
        self.data_dir.mkdir(parents=True, exist_ok=True)
    
    def fetch_from_csv(self, file_path: str) -> pd.DataFrame:
        """
        Load raw NEPSE CSV and perform basic validation.
        Expected columns (based on NEPSE format):
        S.No,Symbol,Conf.,Open,High,Low,Close,LTP,Close - LTP,Close - LTP %,
        VWAP,Vol,Prev. Close,Turnover,Trans.,Diff,Range,Diff %,Range %,VWAP %,
        52 Weeks High,52 Weeks Low
        """
        logger.info(f"Loading data from {file_path}")
        df = pd.read_csv(file_path)
        
        # Validate required columns
        required = ['Symbol', 'Open', 'High', 'Low', 'Close', 'Vol']
        missing = [col for col in required if col not in df.columns]
        if missing:
            raise ValueError(f"Missing columns: {missing}")
        
        # Convert date: if no Date column, create from S.No (assuming daily sequence)
        if 'Date' not in df.columns:
            # Assume data starts from some start_date; for demo, use today - len(df)
            start_date = datetime.now() - pd.Timedelta(days=len(df))
            df['Date'] = pd.date_range(start_date, periods=len(df), freq='B')  # business days
        else:
            df['Date'] = pd.to_datetime(df['Date'])
        
        # Sort by date (important for time series)
        df = df.sort_values('Date')
        
        logger.info(f"Loaded {len(df)} records from {df['Date'].min()} to {df['Date'].max()}")
        return df
    
    def fetch_from_api(self, api_url: str, params: dict = None) -> pd.DataFrame:
        """
        Example API ingestion (hypothetical NEPSE API).
        """
        logger.info(f"Fetching from API: {api_url}")
        resp = requests.get(api_url, params=params)
        resp.raise_for_status()
        data = resp.json()
        # Assume API returns list of dicts
        df = pd.DataFrame(data)
        # Convert date and sort
        df['Date'] = pd.to_datetime(df['date'])
        df = df.sort_values('Date')
        return df
    
    def save_raw(self, df: pd.DataFrame, filename: str = None):
        """
        Save raw data to Parquet (compressed, columnar format).
        """
        if filename is None:
            filename = f"nepse_raw_{datetime.now().strftime('%Y%m%d')}.parquet"
        path = self.data_dir / filename
        df.to_parquet(path, index=False)
        logger.info(f"Saved raw data to {path}")
        return path

# Usage
if __name__ == "__main__":
    ingestor = NEPSEIngestion()
    # Simulate a new CSV arriving daily
    df = ingestor.fetch_from_csv("nepse_latest.csv")
    ingestor.save_raw(df)
```

**Explanation:**

- The `NEPSEIngestion` class abstracts the source of data. Currently, it supports CSV loading but can be extended to APIs.
- CSV loading validates required columns and ensures a proper datetime index. If no `Date` column exists, it creates one based on the assumption of daily business-day data (Monday–Friday). This matches NEPSE’s trading days (Sunday–Thursday), but we use `freq='B'` as a simplification; for exact NEPSE calendar, a custom frequency would be needed.
- Data is saved in **Parquet** format, which is compressed, columnar, and much faster than CSV for repeated reads. This is crucial for downstream feature engineering.
- Logging statements provide visibility into the ingestion process.

---

## **74.3 Feature Engineering Pipeline**

The feature engineering pipeline reads raw data, computes all the features we designed in Part III, and stores them in a **feature store**. The feature store allows us to retrieve features consistently for both training and inference.

We will reuse and extend the `NEPSEFeatureEngineer` class from Chapter 10, but now we will separate the feature computation logic and add a storage layer.

```python
# feature_engineering.py
from typing import List
import pandas as pd
import numpy as np
from pathlib import Path
import logging

logger = logging.getLogger(__name__)

class NEPSEFeatureEngineer:
    """
    Feature engineering for NEPSE data.
    Generates lag, rolling, technical, and domain‑specific features.
    """
    
    def __init__(self):
        self.feature_columns = []  # to be filled after computation
    
    def compute_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Main entry point: compute all features and return a DataFrame with the same index.
        """
        df = df.copy()
        
        # Basic price transformations
        df['Daily_Return'] = df['Close'].pct_change() * 100
        df['Price_Range'] = df['High'] - df['Low']
        df['Price_Change'] = df['Close'] - df['Open']
        df['Upper_Shadow'] = df['High'] - df[['Close', 'Open']].max(axis=1)
        df['Lower_Shadow'] = df[['Close', 'Open']].min(axis=1) - df['Low']
        df['Body_Ratio'] = abs(df['Price_Change']) / (df['Price_Range'] + 1e-6)
        
        # Lag features
        for lag in [1, 2, 3, 5, 10]:
            df[f'Close_Lag_{lag}'] = df['Close'].shift(lag)
            df[f'Return_Lag_{lag}'] = df['Daily_Return'].shift(lag)
            df[f'Volume_Lag_{lag}'] = df['Vol'].shift(lag)
        
        # Rolling window features
        for window in [5, 10, 20]:
            # Trend
            df[f'SMA_{window}'] = df['Close'].rolling(window).mean()
            df[f'EMA_{window}'] = df['Close'].ewm(span=window).mean()
            
            # Volatility
            df[f'Volatility_{window}'] = df['Close'].rolling(window).std()
            df[f'ATR_{window}'] = (df['High'] - df['Low']).rolling(window).mean()
            
            # Volume
            df[f'Volume_SMA_{window}'] = df['Vol'].rolling(window).mean()
            df[f'Volume_Ratio_{window}'] = df['Vol'] / (df[f'Volume_SMA_{window}'] + 1e-6)
            
            # Price range
            df[f'Range_Max_{window}'] = df['High'].rolling(window).max()
            df[f'Range_Min_{window}'] = df['Low'].rolling(window).min()
        
        # Technical indicators
        # RSI
        delta = df['Close'].diff()
        gain = delta.where(delta > 0, 0).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / (loss + 1e-6)
        df['RSI'] = 100 - (100 / (1 + rs))
        
        # MACD
        ema_12 = df['Close'].ewm(span=12).mean()
        ema_26 = df['Close'].ewm(span=26).mean()
        df['MACD'] = ema_12 - ema_26
        df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()
        df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
        
        # Bollinger Bands
        sma_20 = df['Close'].rolling(20).mean()
        std_20 = df['Close'].rolling(20).std()
        df['BB_Upper'] = sma_20 + 2 * std_20
        df['BB_Lower'] = sma_20 - 2 * std_20
        df['BB_Width'] = df['BB_Upper'] - df['BB_Lower']
        df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Width'] + 1e-6)
        
        # NEPSE domain features (circuit breaker proximity, fiscal quarter, etc.)
        # Fiscal calendar (simplified: month to quarter)
        if 'Date' in df.columns:
            df['Month'] = df['Date'].dt.month
            df['Quarter'] = df['Date'].dt.quarter
            # Fiscal year end (mid-July) approximation: July is month 7
            df['Days_to_Fiscal_Year_End'] = (pd.to_datetime(df['Date'].dt.year.astype(str) + '-07-15') - df['Date']).dt.days
        
        # Circuit breaker proximity
        if 'Prev. Close' in df.columns:
            df['Daily_Change_Pct'] = ((df['Close'] - df['Prev. Close']) / df['Prev. Close']) * 100
            df['Upper_Circuit_Proximity'] = 4 - df['Daily_Change_Pct']  # NEPSE first limit 4%
            df['Lower_Circuit_Proximity'] = df['Daily_Change_Pct'] - (-4)
        
        # Volume Z‑score (anomaly detection)
        vol_mean = df['Vol'].rolling(20).mean()
        vol_std = df['Vol'].rolling(20).std()
        df['Volume_Z_Score'] = (df['Vol'] - vol_mean) / (vol_std + 1e-6)
        
        # Drop rows with NaN created by shifts/rolling
        df = df.dropna().reset_index(drop=True)
        
        # Store list of feature columns (exclude metadata and target)
        exclude = ['Date', 'Symbol', 'S.No', 'Conf.', 'Close', 'Prev. Close', 'LTP', 'Turnover', 'Trans.']
        self.feature_columns = [col for col in df.columns if col not in exclude]
        
        logger.info(f"Computed {len(self.feature_columns)} features")
        return df

class FeatureStore:
    """
    Simple feature store that saves/loads feature DataFrames.
    In production, this could be a database (Redis, Feast, etc.).
    """
    def __init__(self, store_dir: str = "./data/features"):
        self.store_dir = Path(store_dir)
        self.store_dir.mkdir(parents=True, exist_ok=True)
    
    def save(self, df: pd.DataFrame, symbol: str, date: datetime):
        """
        Save features for a given symbol and date.
        For simplicity, we append to a single parquet file per symbol.
        """
        file_path = self.store_dir / f"{symbol}_features.parquet"
        # If file exists, read and append, else create new
        if file_path.exists():
            existing = pd.read_parquet(file_path)
            # Avoid duplicates by date
            combined = pd.concat([existing, df], ignore_index=True).drop_duplicates(subset=['Date'], keep='last')
        else:
            combined = df
        combined.to_parquet(file_path, index=False)
        logger.info(f"Saved features for {symbol} to {file_path}")
    
    def load(self, symbol: str, start_date: datetime = None, end_date: datetime = None) -> pd.DataFrame:
        """
        Load features for a symbol, optionally within a date range.
        """
        file_path = self.store_dir / f"{symbol}_features.parquet"
        if not file_path.exists():
            raise FileNotFoundError(f"No features for {symbol}")
        df = pd.read_parquet(file_path)
        if start_date:
            df = df[df['Date'] >= start_date]
        if end_date:
            df = df[df['Date'] <= end_date]
        return df.sort_values('Date')
```

**Explanation:**

- The `NEPSEFeatureEngineer.compute_features` method consolidates all feature creation logic from earlier chapters. It takes a raw DataFrame (with at least `Open`, `High`, `Low`, `Close`, `Vol`, and optionally `Date`) and returns a DataFrame with all engineered features.
- Important: The method **does not** split the data; it simply computes features for whatever data is passed. It also drops rows with NaN values that result from lag and rolling operations. This ensures that downstream models receive a clean matrix.
- The `feature_columns` attribute stores the names of all generated features, which can be used later for selection or model input.
- The `FeatureStore` class provides a simple persistent storage for features. It saves features per symbol in Parquet format. In a real system, you might use a feature store like Feast or Tecton, but this lightweight version suffices for demonstration.
- When saving, we check for duplicates by date and keep the latest, in case the pipeline is rerun for the same day.

---

## **74.4 Model Development**

Now we move to model training. We will:

1. Load historical features from the feature store.
2. Split the data respecting time (no random shuffling).
3. Train a model (we'll use XGBoost as a robust baseline).
4. Register the model and its metadata in MLflow.
5. Save the model for deployment.

```python
# model_training.py
import mlflow
import mlflow.xgboost
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import logging

logger = logging.getLogger(__name__)

class NEPSEForecaster:
    """
    Handles model training, validation, and registration.
    """
    
    def __init__(self, feature_store: FeatureStore, target_col: str = 'Close'):
        self.feature_store = feature_store
        self.target_col = target_col
        self.model = None
        self.feature_columns = None
    
    def prepare_training_data(self, symbol: str, start_date: datetime, end_date: datetime, 
                               feature_cols: List[str] = None):
        """
        Load features and target from feature store.
        """
        df = self.feature_store.load(symbol, start_date, end_date)
        
        if feature_cols is None:
            # Use all features except target and metadata
            exclude = ['Date', 'Symbol', self.target_col]
            feature_cols = [c for c in df.columns if c not in exclude]
        
        X = df[feature_cols]
        y = df[self.target_col]
        
        logger.info(f"Prepared training data: X shape {X.shape}, y shape {y.shape}")
        return X, y, df['Date']
    
    def train_with_cv(self, X, y, dates, n_splits=5):
        """
        Train with time‑series cross‑validation.
        """
        tscv = TimeSeriesSplit(n_splits=n_splits)
        
        cv_scores = []
        models = []
        
        for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            model = xgb.XGBRegressor(
                n_estimators=300,
                max_depth=5,
                learning_rate=0.05,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42
            )
            model.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                early_stopping_rounds=10,
                verbose=False
            )
            
            y_pred = model.predict(X_val)
            mae = mean_absolute_error(y_val, y_pred)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            
            cv_scores.append({'fold': fold, 'mae': mae, 'rmse': rmse})
            models.append(model)
            logger.info(f"Fold {fold}: MAE={mae:.2f}, RMSE={rmse:.2f}")
        
        # Select best model (lowest validation MAE)
        best_idx = np.argmin([s['mae'] for s in cv_scores])
        self.model = models[best_idx]
        logger.info(f"Best model from fold {best_idx} with MAE={cv_scores[best_idx]['mae']:.2f}")
        
        return cv_scores
    
    def train_final(self, X, y):
        """
        Train final model on all data (for production).
        """
        self.model = xgb.XGBRegressor(
            n_estimators=300,
            max_depth=5,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42
        )
        self.model.fit(X, y)
        logger.info("Final model trained on full dataset")
        return self.model
    
    def register_model(self, model_name: str, params: dict, metrics: dict, 
                       feature_cols: List[str], artifact_path: str = "model"):
        """
        Log model to MLflow registry.
        """
        with mlflow.start_run() as run:
            # Log parameters
            for key, value in params.items():
                mlflow.log_param(key, value)
            
            # Log metrics
            for key, value in metrics.items():
                mlflow.log_metric(key, value)
            
            # Log feature list as an artifact
            with open("feature_cols.txt", "w") as f:
                f.write("\n".join(feature_cols))
            mlflow.log_artifact("feature_cols.txt")
            
            # Log the model
            mlflow.xgboost.log_model(self.model, artifact_path, registered_model_name=model_name)
            
            run_id = run.info.run_id
            logger.info(f"Model registered in MLflow run {run_id}")
            return run_id

# Usage example
if __name__ == "__main__":
    # Assume feature store already populated
    store = FeatureStore()
    forecaster = NEPSEForecaster(store)
    
    # Define date range for training (e.g., last 3 years)
    end = datetime.now()
    start = end - timedelta(days=3*365)
    
    X, y, dates = forecaster.prepare_training_data('NEPSE', start, end)
    
    # Cross‑validation
    cv_scores = forecaster.train_with_cv(X, y, dates)
    
    # Compute average metrics
    avg_mae = np.mean([s['mae'] for s in cv_scores])
    avg_rmse = np.mean([s['rmse'] for s in cv_scores])
    
    # Train final model on all data (or optionally on the best fold's training data)
    forecaster.train_final(X, y)
    
    # Register model
    params = {
        'n_estimators': 300,
        'max_depth': 5,
        'learning_rate': 0.05,
        'subsample': 0.8,
        'colsample_bytree': 0.8
    }
    metrics = {'cv_mae': avg_mae, 'cv_rmse': avg_rmse}
    forecaster.register_model('NEPSE_Close_Predictor', params, metrics, X.columns.tolist())
```

**Explanation:**

- We use `TimeSeriesSplit` from scikit‑learn to perform time‑based cross‑validation. This respects the temporal order and prevents future data from leaking into past folds.
- XGBoost is chosen for its robustness with tabular data and ability to handle missing values. We set `early_stopping_rounds` to avoid overfitting.
- The best model from cross‑validation is selected based on validation MAE.
- For production, we then retrain on the full dataset (or you could keep the best fold model). This final model is saved.
- **MLflow** is used for experiment tracking and model registry. We log parameters, metrics, and the list of features used (critical for inference). The model is registered under a name, making it easy to deploy later.
- The code is modular: `prepare_training_data` loads features from the store, so the same pipeline can be used for different symbols or date ranges.

---

## **74.5 Backtesting**

Backtesting evaluates how the model would have performed historically if it had been used for trading. This is more rigorous than simple validation because it simulates a trading strategy (e.g., buy when predicted price increase > 1%, sell when < -1%).

We'll implement a simple backtester that:

- Uses walk‑forward validation (train on expanding window, predict on next day/week).
- Computes trading signals based on predicted returns.
- Calculates performance metrics: total return, Sharpe ratio, max drawdown.

```python
# backtesting.py
import pandas as pd
import numpy as np
from typing import Dict, Tuple

class NEPSEBacktester:
    """
    Walk‑forward backtesting with a simple trading strategy.
    """
    
    def __init__(self, model, feature_columns: List[str], initial_capital: float = 100000):
        self.model = model
        self.feature_columns = feature_columns
        self.initial_capital = initial_capital
    
    def run(self, X: pd.DataFrame, y_true: pd.Series, dates: pd.Series) -> Dict:
        """
        Simulate trading day by day.
        Strategy: If predicted return > threshold, buy; if < -threshold, sell.
        For simplicity, we assume we can only be long or flat (no short).
        """
        # Ensure X contains only the features the model expects
        X = X[self.feature_columns]
        
        # Get predictions
        y_pred = self.model.predict(X)
        
        # Compute predicted returns (as percentage of current price)
        # Since model predicts absolute close price, we convert to return
        pred_return = (y_pred / y_true.values - 1) * 100
        
        # Generate signals: 1 = buy, -1 = sell (go flat), 0 = hold
        # Simple threshold: 1% predicted return
        signals = np.zeros(len(X))
        signals[pred_return > 1.0] = 1     # buy
        signals[pred_return < -1.0] = -1   # sell (go flat)
        
        # Simulate portfolio
        portfolio = pd.DataFrame(index=dates)
        portfolio['price'] = y_true.values
        portfolio['signal'] = signals
        portfolio['position'] = portfolio['signal'].replace({-1: 0, 1: 1, 0: np.nan}).ffill().fillna(0)
        portfolio['returns'] = portfolio['price'].pct_change()
        portfolio['strategy_returns'] = portfolio['position'].shift(1) * portfolio['returns']
        
        # Calculate cumulative wealth
        portfolio['cumulative_market'] = (1 + portfolio['returns']).cumprod()
        portfolio['cumulative_strategy'] = (1 + portfolio['strategy_returns']).cumprod() * self.initial_capital
        
        # Performance metrics
        total_return = (portfolio['cumulative_strategy'].iloc[-1] / self.initial_capital - 1) * 100
        sharpe = portfolio['strategy_returns'].mean() / portfolio['strategy_returns'].std() * np.sqrt(252)
        max_drawdown = (portfolio['cumulative_strategy'] / portfolio['cumulative_strategy'].cummax() - 1).min()
        
        metrics = {
            'total_return_pct': total_return,
            'sharpe_ratio': sharpe,
            'max_drawdown_pct': max_drawdown * 100,
            'num_trades': (signals != 0).sum(),
        }
        
        return metrics, portfolio

# Usage within the training pipeline
def backtest_model(model, X, y, dates):
    bt = NEPSEBacktester(model, X.columns.tolist())
    metrics, portfolio = bt.run(X, y, dates)
    print("Backtest Metrics:")
    for k, v in metrics.items():
        print(f"  {k}: {v:.2f}")
    return metrics, portfolio
```

**Explanation:**

- The backtester uses the trained model to generate predictions on historical data, then simulates a simple trading strategy. This is a **walk‑forward** simulation because we are using the model as if it were making predictions day by day.
- To avoid look‑ahead, we must ensure that the model was trained only on data prior to each prediction. Our `run` method assumes that the model was trained on data ending before the first date in `dates`. This is true if we use a walk‑forward training scheme (not shown here but can be implemented with a loop).
- The strategy: if predicted return > 1%, buy (go long); if predicted return < -1%, sell (go flat). We hold the position until a new signal arrives.
- Metrics: total return (percentage), Sharpe ratio (risk‑adjusted return), maximum drawdown, and number of trades.
- The `portfolio` DataFrame is returned for further analysis (e.g., plotting equity curve).

---

## **74.6 Production Deployment**

In production, we need to serve predictions both in **batch** mode (e.g., daily after market close) and **real‑time** (e.g., via API). We will build two services:

1. **Batch prediction service**: runs daily, loads new features, generates predictions, and stores them in a database or sends them to a dashboard.
2. **REST API**: allows on‑demand prediction for a given stock.

### **74.6.1 Batch Prediction Service**

```python
# batch_predictor.py
import pandas as pd
from datetime import datetime, timedelta
import mlflow.pyfunc
import logging

logger = logging.getLogger(__name__)

class BatchPredictor:
    """
    Daily batch prediction job.
    """
    
    def __init__(self, model_name: str, model_stage: str = "Production", 
                 feature_store: FeatureStore = None):
        """
        Load model from MLflow registry.
        """
        self.model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_stage}")
        self.feature_store = feature_store or FeatureStore()
        # Retrieve feature list (should be stored as artifact during training)
        # In practice, you'd store this in the model's metadata.
        self.feature_columns = self._load_feature_columns(model_name)
    
    def _load_feature_columns(self, model_name):
        # Dummy implementation – in reality, fetch from MLflow artifact
        # For now, assume we saved a file with the model
        # ...
        return ['Close_Lag_1', 'SMA_20', 'RSI', ...]  # placeholder
    
    def predict_for_date(self, symbol: str, date: datetime) -> float:
        """
        Generate prediction for a single date using the latest available features.
        """
        # Load features for the symbol up to the day before (since we can't know today's close yet)
        # For simplicity, assume we have features up to date-1
        df = self.feature_store.load(symbol, end_date=date - timedelta(days=1))
        if df.empty:
            raise ValueError(f"No features available for {symbol} before {date}")
        latest = df.iloc[-1:]
        X = latest[self.feature_columns]
        pred = self.model.predict(X)[0]
        return pred
    
    def run_daily(self, symbols: List[str], target_date: datetime = None):
        """
        Run batch prediction for all symbols on target_date.
        """
        if target_date is None:
            target_date = datetime.now().date()
        results = []
        for symbol in symbols:
            try:
                pred = self.predict_for_date(symbol, target_date)
                results.append({'symbol': symbol, 'date': target_date, 'predicted_close': pred})
                logger.info(f"Predicted {symbol} close: {pred:.2f}")
            except Exception as e:
                logger.error(f"Failed for {symbol}: {e}")
        # Save results (to CSV, database, etc.)
        df = pd.DataFrame(results)
        df.to_csv(f"predictions_{target_date}.csv", index=False)
        logger.info(f"Batch predictions saved for {target_date}")
        return df
```

### **74.6.2 REST API Service**

We'll build a simple REST API using FastAPI.

```python
# api_service.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from datetime import date
import pandas as pd
import mlflow.pyfunc
import logging

app = FastAPI(title="NEPSE Prediction API")

# Global model and feature store (loaded at startup)
model = None
feature_store = None
feature_columns = None

class PredictionRequest(BaseModel):
    symbol: str
    date: date  # The date for which we want prediction (usually today)

class PredictionResponse(BaseModel):
    symbol: str
    date: date
    predicted_close: float
    model_version: str

@app.on_event("startup")
def load_model():
    global model, feature_store, feature_columns
    model = mlflow.pyfunc.load_model("models:/NEPSE_Close_Predictor/Production")
    feature_store = FeatureStore()
    # Load feature columns from somewhere (e.g., from model's run artifacts)
    # Here we hardcode for demonstration
    feature_columns = ['Close_Lag_1', 'SMA_20', 'RSI', 'Volume_Z_Score']  # example
    logging.info("Model loaded successfully")

@app.get("/health")
def health():
    return {"status": "healthy"}

@app.post("/predict", response_model=PredictionResponse)
def predict(request: PredictionRequest):
    # Get features for the symbol up to the day before request.date
    # (since we cannot use future data)
    end_date = request.date - pd.Timedelta(days=1)
    df = feature_store.load(request.symbol, end_date=end_date)
    if df.empty:
        raise HTTPException(status_code=404, detail="No features found for symbol/date")
    latest = df.iloc[-1:]
    X = latest[feature_columns]
    pred = model.predict(X)[0]
    return PredictionResponse(
        symbol=request.symbol,
        date=request.date,
        predicted_close=float(pred),
        model_version="1.0.0"  # in practice, fetch from model metadata
    )
```

**Explanation:**

- The batch predictor loads the model from MLflow (by stage, e.g., "Production") and uses the feature store to retrieve the latest features. It then predicts the next day's close price.
- The REST API is built with FastAPI, which automatically generates OpenAPI documentation. It loads the model at startup and provides an endpoint `/predict` that accepts a symbol and date.
- Both services ensure that no future data is used: they only access features up to the day before the prediction date.
- In production, you would containerise these services (Docker) and deploy them using Kubernetes or a cloud platform.

---

## **74.7 Monitoring**

Monitoring is crucial to detect when the model's performance degrades. We'll implement:

- **Prediction monitoring**: store predictions and actuals, compute daily/weekly error metrics.
- **Data drift monitoring**: compare distributions of key features between training and recent data using statistical tests (e.g., Kolmogorov–Smirnov).
- **System health monitoring**: track API latency, error rates, and data freshness.

We can reuse the alerting framework from Chapter 73 to notify when metrics exceed thresholds.

```python
# monitoring.py
import pandas as pd
from scipy.stats import ks_2samp
import numpy as np
from datetime import datetime, timedelta
import logging

logger = logging.getLogger(__name__)

class ModelMonitor:
    """
    Monitors prediction accuracy and data drift.
    """
    
    def __init__(self, feature_store: FeatureStore, 
                 prediction_store_path: str = "./data/predictions.parquet"):
        self.feature_store = feature_store
        self.prediction_store_path = prediction_store_path
        self.alerts = []  # or integrate with AlertManager
    
    def log_prediction(self, symbol: str, date: datetime, predicted: float, actual: float = None):
        """
        Store a prediction; actual can be updated later when known.
        """
        df = pd.DataFrame([{
            'symbol': symbol,
            'date': date,
            'predicted': predicted,
            'actual': actual,
            'timestamp': datetime.now()
        }])
        if Path(self.prediction_store_path).exists():
            existing = pd.read_parquet(self.prediction_store_path)
            df = pd.concat([existing, df], ignore_index=True)
        df.to_parquet(self.prediction_store_path, index=False)
    
    def update_actual(self, symbol: str, date: datetime, actual: float):
        """
        Update the actual value for a past prediction.
        """
        df = pd.read_parquet(self.prediction_store_path)
        mask = (df['symbol'] == symbol) & (df['date'] == date)
        df.loc[mask, 'actual'] = actual
        df.to_parquet(self.prediction_store_path, index=False)
    
    def compute_daily_errors(self, date: datetime = None):
        """
        Compute error metrics for predictions where actual is available.
        """
        df = pd.read_parquet(self.prediction_store_path)
        if date is not None:
            df = df[df['date'] == date]
        df = df.dropna(subset=['actual'])
        if df.empty:
            return {}
        errors = df['predicted'] - df['actual']
        mae = np.abs(errors).mean()
        rmse = np.sqrt((errors**2).mean())
        mape = (np.abs(errors) / df['actual']).mean() * 100
        return {'mae': mae, 'rmse': rmse, 'mape': mape}
    
    def detect_data_drift(self, symbol: str, reference_start: datetime, 
                          reference_end: datetime, current_start: datetime,
                          current_end: datetime, features: List[str], 
                          p_threshold: float = 0.05):
        """
        Use Kolmogorov‑Smirnov test to detect drift in feature distributions.
        """
        ref_df = self.feature_store.load(symbol, reference_start, reference_end)
        curr_df = self.feature_store.load(symbol, current_start, current_end)
        
        drift_report = {}
        for f in features:
            if f not in ref_df or f not in curr_df:
                continue
            ref_vals = ref_df[f].dropna()
            curr_vals = curr_df[f].dropna()
            if len(ref_vals) == 0 or len(curr_vals) == 0:
                continue
            ks_stat, p_value = ks_2samp(ref_vals, curr_vals)
            drift_detected = p_value < p_threshold
            drift_report[f] = {
                'ks_statistic': ks_stat,
                'p_value': p_value,
                'drift_detected': drift_detected
            }
            if drift_detected:
                logger.warning(f"Drift detected in feature '{f}' (p={p_value:.4f})")
                # Optionally trigger an alert
        return drift_report
```

**Explanation:**

- `ModelMonitor` logs predictions and, when actual values become available (e.g., next day's close), updates them.
- `compute_daily_errors` calculates MAE, RMSE, and MAPE for any day where actuals exist.
- `detect_data_drift` compares feature distributions between a reference period (typically training period) and a recent window using the two‑sample Kolmogorov–Smirnov test. A low p‑value indicates that the distributions have changed, which may signal that the model's assumptions are no longer valid.
- This drift detection can be scheduled to run weekly and integrated with the alert manager from Chapter 73 to notify the team.

---

## **74.8 Performance Analysis**

After running the system for a period, we need to analyse its performance. This includes:

- Comparing predicted vs actual prices over time.
- Analysing error distribution and identifying systematic biases (e.g., always under‑predicting during bull runs).
- Segmenting performance by market conditions (high/low volatility, circuit breaker days, etc.).
- Reviewing alert history and incident response.

We'll create a simple analysis notebook that loads prediction logs and feature store data.

```python
# analysis.py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def load_prediction_logs(path: str = "./data/predictions.parquet") -> pd.DataFrame:
    df = pd.read_parquet(path)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(['symbol', 'date'])
    df['error'] = df['predicted'] - df['actual']
    df['abs_error'] = df['error'].abs()
    df['error_pct'] = (df['error'] / df['actual']) * 100
    return df

def plot_error_over_time(df, symbol='NEPSE'):
    sym_df = df[df['symbol'] == symbol]
    plt.figure(figsize=(12, 5))
    plt.plot(sym_df['date'], sym_df['error'], label='Prediction Error')
    plt.axhline(y=0, color='red', linestyle='--')
    plt.title(f'Prediction Errors Over Time for {symbol}')
    plt.xlabel('Date')
    plt.ylabel('Error (NPR)')
    plt.legend()
    plt.grid(True)
    plt.show()

def error_distribution(df):
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    sns.histplot(df['error'], bins=50, kde=True)
    plt.title('Error Distribution')
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df['error'])
    plt.title('Error Boxplot')
    plt.tight_layout()
    plt.show()

def performance_by_volatility(df, feature_store, symbol='NEPSE'):
    # Merge with volatility feature
    vol_df = feature_store.load(symbol, df['date'].min(), df['date'].max())
    vol_df = vol_df[['Date', 'Volatility_20']]
    merged = pd.merge(df, vol_df, left_on='date', right_on='Date', how='inner')
    # Create volatility buckets
    merged['vol_bucket'] = pd.cut(merged['Volatility_20'], bins=4, labels=['Low', 'Med-Low', 'Med-High', 'High'])
    perf = merged.groupby('vol_bucket')['abs_error'].agg(['mean', 'std', 'count'])
    return perf
```

**Explanation:**

- `load_prediction_logs` reads the stored predictions and adds error columns.
- `plot_error_over_time` visualises errors to spot trends or seasonal patterns.
- `error_distribution` helps understand if errors are normally distributed or skewed.
- `performance_by_volatility` joins with the feature store to get a volatility measure and then segments error by volatility regime. This can reveal if the model struggles in high‑volatility periods.

---

## **74.9 Lessons Learned**

Running a real‑world prediction system teaches valuable lessons. Here are some we would likely encounter with the NEPSE system:

1. **Data quality is paramount** – Missing values, corporate actions (splits, dividends), and data entry errors can severely degrade performance. Implement robust validation and anomaly detection.
2. **Feature selection must be revisited** – Some features that worked well historically may lose predictive power due to market regime changes. Regular feature importance analysis and periodic retraining are essential.
3. **Overfitting to noise** – In a volatile market like NEPSE, models can easily fit to random fluctuations. Walk‑forward validation and simple models (like linear models) sometimes outperform complex ones.
4. **Alert fatigue** – Too many alerts lead to ignored notifications. Fine‑tune thresholds and cooldowns; use severity levels wisely.
5. **Infrastructure matters** – A slow API or unreliable batch job can ruin user trust. Invest in monitoring and scalable deployment.
6. **Regulatory compliance** – In financial applications, you must document everything: data provenance, feature definitions, model versions, and decision rationale. This is not optional.

---

## **74.10 Future Improvements**

The system we built is functional but can be enhanced in many ways:

- **Incorporate alternative data** – News sentiment, social media trends, macroeconomic indicators.
- **Multi‑step forecasting** – Predict not only next day but also 5‑day ahead.
- **Deep learning models** – LSTM or Transformer‑based models might capture longer‑range dependencies.
- **Automated retraining** – Use drift detection to trigger retraining automatically.
- **A/B testing** – Deploy two models simultaneously and compare performance with live trades.
- **Explainability** – Integrate SHAP or LIME to provide explanations for each prediction (important for trader trust).
- **Real‑time streaming** – Move from daily batch to intra‑day predictions using tick data.

---

## **Chapter Summary**

In this final chapter, we assembled all the components of a complete time‑series prediction system, using NEPSE as our guiding example. We covered:

- The overall architecture, separating data, features, models, prediction, and monitoring layers.
- A robust data ingestion pipeline that handles CSV sources and stores raw data in Parquet.
- A feature engineering pipeline that computes a wide array of features and persists them in a feature store.
- Model development with time‑series cross‑validation and MLflow tracking.
- Backtesting that simulates a simple trading strategy.
- Production deployment as a batch job and a REST API.
- Monitoring for prediction accuracy and data drift, integrated with the alerting system from Chapter 73.
- Performance analysis techniques to evaluate model behaviour over time.
- Real‑world lessons and a roadmap for future enhancements.

You now have a blueprint for building your own time‑series prediction system, whether for stocks, retail sales, weather, or any other domain. The principles and code patterns are transferable; only the domain‑specific features need adaptation.

Congratulations on completing this journey through the **Time‑Series Prediction System** handbook. May your predictions be accurate and your systems robust!

---

**End of Chapter 74**